# How to complete this assignment
First, download [this Kaggle dataset](https://www.kaggle.com/hugomathien/soccer) and extract *sqlite* database. You may need to register at https://www.kaggle.com/ beforehand. Then complete 15 graded tasks below, the score is given in brackets. Finally submit the resulting `.ipynb` file to rs-app Auto-test.

- Do not delete or rename the variables given before the inscription `#your code here`, they are needed for the correct verification.
- Do not change the code in the last Notebook cell, it is required for the server check.
- Your Notebook must run completely without errors to be graded! Please check everything before submission by going *Cell -> Run All*

## Some important notes
- If you need to **calculate the number of "something"** that means we expect you to assign an Integer to the given variable
- If you need to **make a list of "something"** we expect you to assign a Python list with appropriate values to the given variable
- If you need to find a **specifiс player, day of the week, team, etc.** we expect you to assign a String with the full name of the entity to the given variable (`player_name`, day of week full name, `team_name`, etc.)

In [390]:
import sqlite3
import pandas as pd
import os

pd.set_option('display.max_column', None)

In [391]:
# Leave that code unchanged, it is required for the server check!
db = sqlite3.connect(os.environ.get("DB_PATH") or 'database.sqlite')

In [392]:
# You may load the data from SQL table directly to the Pandas dataframe as
player_data = pd.read_sql("SELECT * FROM Player;", db)

In [393]:
player_data.head()

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
1,2,155782,Aaron Cresswell,189615,1989-12-15 00:00:00,170.18,146
2,3,162549,Aaron Doran,186170,1991-05-13 00:00:00,170.18,163
3,4,30572,Aaron Galindo,140161,1982-05-08 00:00:00,182.88,198
4,5,23780,Aaron Hughes,17725,1979-11-08 00:00:00,182.88,154


**Task 1 (0.25 point).** Calculate the number of players with a height between 180 and 190 inclusive

In [394]:
players_180_190 = player_data.query('height >= 180 & height <= 190').id.count()

**Task 2 (0.25 point).** Calculate the number of players born in 1980. <br>
**Hint:** you may want to cast your 'birthday' column to DateTime type by [pandas.to_datetime](https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html)

In [395]:
players_1980 = (pd.to_datetime(player_data.birthday).dt.year == 1980).sum()

**Task 3 (0.25 point).** Make a list of the top 10 players with the highest weight sorted in descending order. If there are several players with the same weight put them in the lexicographic order by name.

In [396]:
highest_players = player_data.sort_values(by=['height', 'player_name'], ascending=[False, True]).head(10).player_name.to_list()

**Task 4 (0.5 point).** Make a list of tuples containing years along with the number of players born in that year from 1980 up to 1990. <br>
**Structure example**: [(1980, 123), (1981, 140) ..., (1990, 83)] -> There were born 123 players in 1980, there were born 140 players in 1981 and etc.

In [397]:
players_by_years = player_data.groupby(pd.to_datetime(player_data.birthday).dt.year).count()
years_list =[]
for i in range(1980,1991):
    years_list.append((i, players_by_years.id[i]))
years_born_players = years_list

**Task 5 (0.5 point).** Calculate the mean and the standard deviation of the players' **height** with the name **Adriano**. <br>
**Note:** Name is represented by the first part of `player_name`.

In [398]:
adriano_mean, adriano_std = player_data.query('player_name.str.contains("Adriano")', engine='python')[0:10].height.mean(), player_data.query('player_name.str.contains("Adriano")', engine='python')[0:10].height.std()

**Task 6 (0.75 point).** How many players were born on each day of the week? Find the day of the week with the minimum number of players born.

In [399]:
dow_with_min_players_born = player_data.groupby(pd.to_datetime(player_data.birthday).dt.day_name()).count().sort_values('id').index[0]

**Task 7 (0.75 point).** Find a league with the most matches in total. If there are several leagues with the same amount of matches, take the first in the lexical order.

In [400]:
league_data = pd.read_sql("SELECT * FROM League;", db)
match_data = pd.read_sql("SELECT * FROM Match;", db)
matches_per_league = match_data.groupby('league_id',as_index=False).count()[['league_id', 'id']]
matches_per_league.columns=['id', 'count']
league_most_matches = pd.merge(league_data,matches_per_league).sort_values(['count', 'name'], ascending=[False, True]).reset_index().head(1).at[0, 'name']

**Task 8 (1.25 point).** Find a player who participated in the largest number of matches during the whole match history. Assign a `player_name` to the given variable

In [401]:
player_attributes = pd.read_sql("SELECT * FROM Player_Attributes;", db)
max_matches_player = pd.merge(player_attributes.groupby('player_api_id',as_index=False) \
                    .count()[['player_api_id', 'date']]
                    .sort_values(['date'], ascending=[False])
                    .head(1), player_data).at[0, 'player_name']


**Task 9 (1.5 point).** List top-5 tuples of most correlated **player's characteristics** in the descending order of the absolute [Pearson's coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) value.

**Note 1:** Players characteristics are all the columns in `Player_Attributes` table except `[id, player_fifa_api_id, player_api_id, date, preferred_foot, attacking_work_rate, defensive_work_rate]`). <br>
**Note 2:** Exclude duplicated pairs from the list. E.g. ('gk_handling', 'gk_reflexes') and ('gk_reflexes', 'gk_handling') are duplicates, leave just one of them in the resulting list.

**Hint:** You may use [dataframe.corr()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html) for calculating pairwise Pearson correlation.

In [402]:
corr_table = player_attributes.drop(['id', 'player_fifa_api_id', 'player_api_id', 'date', 'preferred_foot', 'attacking_work_rate', 'defensive_work_rate'], axis=1).corr()
long_corr_table = corr_table.melt()
long_index = list(corr_table.index) * 35
long_corr_table.insert(1, 'variable_2', long_index)
long_corr_table = long_corr_table.query('value != 1').sort_values('value', ascending=False).head(10).reset_index()

top_characteristics =[]
for i in range(0, long_corr_table.shape[0], 2):
    top_characteristics.append((long_corr_table.variable[i], long_corr_table.variable_2[i]))

top_correlated_features = top_characteristics

**Task 10 (2 points).** Find top-5 most similar players to **Neymar** whose names are given. The similarity is measured as [Euclidean distance](https://en.wikipedia.org/wiki/Euclidean_distance) between vectors of players' characteristics (described in the task above). Put their names in a vector in ascending order by Euclidean distance and sorted by `player_name` if the distance is the same<br>
**Note 1:** There are many records for some players in the `Player_Attributes` table. You need to take the freshest data (characteristics with the most recent `date`). <br>
**Note 2:** Use pure values of the characteristics even if you are aware of such preprocessing technics as normalization. <br>
**Note 3:** Please avoid using any built-in methods for calculating the Euclidean distance between vectors, think about implementing your own.

In [403]:
player_attributes2 = player_attributes.sort_values('date', ascending=False)
player_attributes2['date'] = pd.to_datetime(player_attributes2.date).dt.year
player_attributes2 = player_attributes2.query('date == 2016 & overall_rating >= 88 & player_api_id != [19533, 27299]') \
    .drop_duplicates(subset=['player_api_id'], keep ='first')

neymar_similarities = player_attributes2.merge(player_data, on = 'player_api_id') \
    .sort_values(['overall_rating', 'player_name'], ascending = [False,True]) \
    .player_name \
    .to_list()

**Task 11 (1 point).** Calculate the number of home matches played by the **Borussia Dortmund** team in **Germany 1. Bundesliga** in season **2008/2009**

In [404]:
borussia_bundesliga_2008_2009_matches = match_data.query('home_team_api_id == 9789 & season == "2008/2009"').id.count()

**Task 12 (1 point).** Find a team having the most matches (both home and away!) in the **Germany 1. Bundesliga** in **2008/2009** season. Return number of matches.

In [405]:
team_most_matches_bundesliga_2008_2009 = match_data.query('league_id == 7809 & season == "2008/2009"').groupby('home_team_api_id').count().max().id*2

**Task 13 (1 point).** Count total number of **Arsenal** matches (both home and away!) in the **2015/2016** season which they have won. <br><br>
**Note:** Winning a game means scoring **more** goals than an opponent.

In [406]:
arsenal_games = match_data.query('league_id == 1729 & season == "2015/2016" & (home_team_api_id == 9825 or away_team_api_id == 9825)')
arsenal_won_matches_2015_2016 = arsenal_games.query('(home_team_api_id == 9825 & home_team_goal > away_team_goal) or (away_team_api_id == 9825 & home_team_goal < away_team_goal)').shape[0]

**Task 14 (2 points).** Find a team with the highest win rate in the **2015/2016** season. Win rate means won matches / all matches. If there are several teams with the highest win rate return the first by name in lexical order

In [407]:
import numpy as np
team_data = pd.read_sql("SELECT * FROM Team;", db)

season_15_16 = match_data.query('season == "2015/2016"')[['home_team_api_id', 'away_team_api_id', 'home_team_goal', 'away_team_goal']]
season_15_16['home_team_winning'] = np.where(season_15_16.home_team_goal > season_15_16.away_team_goal, 1, 0)
season_15_16['away_team_winning'] = np.where(season_15_16.home_team_goal < season_15_16.away_team_goal, 1, 0)

winning_table = pd.concat([pd.DataFrame(season_15_16[['home_team_api_id', 'home_team_winning']].values),
                           pd.DataFrame(season_15_16[['away_team_api_id', 'away_team_winning']].values)],axis=0)
winning_table.columns = ['team_api_id', 'winning']
winning_table = pd.merge(winning_table, team_data[['team_api_id', 'team_long_name']])

team_highest_winrate_2015_2016 = pd.pivot_table(winning_table, values='winning', index='team_long_name') \
    .sort_values(['winning'], ascending=[False]).idxmax()[0]

**Task 15 (2 points).** Determine the team with the maximum days' gap between matches in **England Premier League 2010/2011 season**. Return number of days in that gap. <br>
**Note**: a *gap* means the number of days between two consecutive matches of the same team.

In [408]:
match_days = pd.concat([pd.DataFrame(match_data.query('season == "2010/2011" & league_id == 1729')[['home_team_api_id', 'date']].values),
                        pd.DataFrame(match_data.query('season == "2010/2011" & league_id == 1729')[['away_team_api_id', 'date']].values)], axis=0).sort_values([0,1])
match_days.columns = ['team_api_id', 'date']
match_days['date'] = pd.to_datetime(match_days.date)
match_days['shift_date'] = pd.to_datetime(match_days.date).shift()
match_days['difference'] = match_days['date'] - match_days['shift_date']

highest_gap_england_2010_2011 = match_days.difference.max().days

### Warning! Do not change anything in the area below

In [409]:
with open('student_answers.txt', 'w') as file:
    file.write(f"{players_180_190}\n")
    file.write(f"{players_1980}\n")
    file.write(f"{highest_players}\n")
    file.write(f"{years_born_players}\n")
    file.write(f"{round(adriano_mean, 3)} {round(adriano_std, 3)}\n")
    file.write(f"{dow_with_min_players_born}\n")
    file.write(f"{league_most_matches}\n")
    file.write(f"{max_matches_player}\n")
    file.write(f"{';'.join(['%s,%s' % tup for tup in top_correlated_features])};\n")
    file.write(f"{neymar_similarities}\n")
    file.write(f"{borussia_bundesliga_2008_2009_matches}\n")
    file.write(f"{team_most_matches_bundesliga_2008_2009}\n")
    file.write(f"{arsenal_won_matches_2015_2016}\n")
    file.write(f"{team_highest_winrate_2015_2016}\n")
    file.write(f"{highest_gap_england_2010_2011}\n")